In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from random import randint
import librosa
import librosa.display
import IPython.display as ipd
from IPython.display  import Audio
from sklearn import preprocessing
import seaborn as sns
import missingno as msno
import plotly.graph_objects as go

### Feature Selection

In [2]:
# Código Nicolás: he tratado de hacer que se extraigan las caracteríscas en ventanas de 10 ms
def normalize(x, axis = 0):
    return preprocessing.minmax_scale(x, axis = axis)

def extrae_var(cancion):

    # Obtener la muestra de audio (samples) y el tamaño de la ventana(sr)
    samples, sr = librosa.load(cancion, sr=None)

    features = [cancion]

    # Configuración para ventanas de 20 ms
    frame_length = 1024  # 20 ms en muestras (20 y algo para que sea múltiplo de 2)
    hop_length = 512  # Salto de 10 ms (10 y algo para que sea múltiplo de 2)

    # Tempo
    onset_env = librosa.onset.onset_strength(y=samples, sr=sr, hop_length = hop_length, n_fft=frame_length)
    tempo, _ = librosa.beat.beat_track(onset_envelope=onset_env, sr=sr)
    features.append(tempo)

    # Espectrograma
    spectrogram = np.abs(librosa.stft(samples, hop_length=hop_length, n_fft=frame_length))
    features.append(np.mean(spectrogram))
    

    # Mel Spectogram
    mel_spectogram = librosa.feature.melspectrogram(y=samples, sr=sr, n_fft=frame_length, hop_length=hop_length)
    features.append(np.mean(mel_spectogram))

    # Media de la energía en cada ventana de 10 ms
    rms = librosa.feature.rms(y=samples, hop_length = hop_length, frame_length=frame_length)[0]
    features.append(np.mean(rms))

    # Centroides espectrales
    spectral_centroids = librosa.feature.spectral_centroid(y=samples, sr=sr, hop_length = hop_length, n_fft=frame_length)[0]
    features.append(np.mean(spectral_centroids))

    # Ancho de banda de frecuencia
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=samples, sr=sr, hop_length = hop_length, n_fft=frame_length)[0]
    features.append(np.mean(spectral_bandwidth))

    # Spectral Rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y=samples + 0.01, sr=sr, hop_length = hop_length, n_fft=frame_length)[0]
    features.append(np.mean(spectral_rolloff))

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(samples), sr=sr)
    features.append(np.mean(tonnetz))

    # ZCR
    zero_crossings= librosa.feature.zero_crossing_rate(y=samples, frame_length=frame_length, hop_length=hop_length)[0]
    features.append(np.mean(zero_crossings))

    # Cromatica
    cromatica = librosa.feature.chroma_stft(y=samples, sr=sr, hop_length = hop_length, n_fft=frame_length)[0]
    features.append(np.mean(cromatica))

    # TEMPOGRAMA
    tempograma = librosa.feature.tempogram(onset_envelope = onset_env, sr = sr, hop_length = hop_length, win_length=frame_length)[0]

    # Auto-correlation
    auto_c = librosa.autocorrelate(onset_env, max_size = tempograma.shape[0])
    auto_c = librosa.util.normalize(auto_c)
    features.append(np.mean(auto_c))

    #Pulse
    pulse = librosa.beat.plp(onset_envelope=onset_env, sr=sr, hop_length=hop_length, win_length=frame_length)
    features.append(np.mean(pulse))

    # HArmonic, percussive

    D=librosa.stft(samples, hop_length=hop_length, n_fft=frame_length)
    D_harmonic, D_percussive = librosa.decompose.hpss(D)
    features.append(np.mean(np.abs(D_harmonic)))
    features.append(np.mean(np.abs(D_percussive)))

    # MFCCS
    mfccs = librosa.feature.mfcc(y = samples, sr = sr, n_mfcc=12)
    
    columnas = ['archivo', 'tempo', 'spectrogram', 'mel_spectrogram', 'rms', 'spectral_centroids', 'spectral_bandwith', 'spectral_rf', 'tonnetz', 'zero_crossings', "cromatica", "auto-correlation","pulse","harmonic","percussive"]

    for x in range(1, 13):
        columnas.append(f'mfcc_{x}')

    features.extend([np.mean(m) for m in mfccs])

    df = pd.DataFrame(columns=columnas)

    # Fila de datos
    df.loc[len(df)] = features

    return df

In [3]:
df_t=pd.DataFrame()
i=0
for subcarpeta in os.listdir('data'):
    print("Procesando canciones de:", subcarpeta)
    for archivo in os.listdir('data/'+subcarpeta):
        if archivo.endswith('.mp3'):
            cancion='data/'+subcarpeta +'/'+archivo
            df=extrae_var(cancion)
            df['Genero_label']=i #genera una etiqueta numerica para entrenamiento de modelo
            df['Genero']=subcarpeta #genera etiqueta de nombre para las resententaciones
            df_t=pd.concat([df_t,df])
            
    i=i+1
df_t.reset_index(drop=True,inplace=True)

Procesando canciones de: jazz
Procesando canciones de: opera
Procesando canciones de: rap
Procesando canciones de: reggae
Procesando canciones de: rock


In [4]:
df_t

,archivo,tempo,spectrogram,mel_spectrogram,rms,spectral_centroids,spectral_bandwith,spectral_rf,tonnetz,zero_crossings,...,mfcc_13,mfcc_14,mfcc_15,mfcc_16,mfcc_17,mfcc_18,mfcc_19,mfcc_20,Genero_label,Genero
0,data/jazz/Body And Soul (2008 Remastered)_4LnE...,123.046875,0.035016,0.001951,0.092547,1092.073000,1367.988176,1894.921875,0.024475,0.058594,...,-5.664622,-2.206847,-5.138870,-0.625633,-1.046974,-0.461257,-2.626390,-1.119201,0,jazz
1,data/jazz/Ella Fitzgerald - How High The Moon ...,135.999178,0.257868,0.005483,0.092986,2959.370186,2956.973763,6804.492188,-0.009210,0.148438,...,-3.608733,4.722158,-1.640425,1.422542,-0.563128,3.611902,-3.734178,2.617666,0,jazz
2,data/jazz/Ella Fitzgerald - Mack The Knife (Li...,123.046875,0.196014,0.004419,0.133279,2219.312036,2621.020585,4823.437500,-0.002739,0.095703,...,-5.274746,0.842855,-3.663515,2.292180,-3.587532,2.270871,-4.524094,0.785091,0,jazz
3,data/jazz/Fats Waller - Ain't Misbehavin' (Aud...,129.199219,0.032406,0.000634,0.062436,1330.594309,1562.213603,2497.851562,0.026911,0.068359,...,-6.734543,-4.214897,-8.875837,-5.021723,-6.187879,-4.756309,-4.550775,-6.682107,0,jazz
4,data/jazz/Fly Me To The Moon (2008 Remastered)...,123.046875,0.067773,0.001232,0.093297,1651.902160,2226.481295,3402.246094,0.009794,0.062500,...,-1.189884,2.183551,-2.938324,3.423477,-2.259246,-0.117523,-1.375480,0.565481,0,jazz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,data/rock/Rock and Roll - Remaster.mp3,123.046875,0.517485,0.032561,0.189727,2483.290973,2540.707479,5297.167969,-0.006046,0.125000,...,-3.230034,5.516514,-3.415281,3.273184,-0.645325,-1.956044,-7.267938,-3.967822,4,rock
96,data/rock/The Jimi Hendrix Experience - Purple...,112.347147,0.321128,0.029975,0.184136,2221.238126,2095.532570,3962.109375,0.018674,0.134766,...,-5.009894,-5.284219,0.000000,1.201538,-13.716368,-1.130577,3.789937,-1.883433,4,rock
97,data/rock/The Rolling Stones - (I Cant Get No)...,135.999178,0.205729,0.012735,0.132289,2067.506564,2045.510712,3962.109375,0.020824,0.105469,...,-3.178892,1.534376,-2.352422,0.000000,-2.894109,-1.296950,-3.494431,3.627535,4,rock
98,data/rock/The Rolling Stones - Gimme Shelter (...,117.453835,0.193751,0.013184,0.167720,1637.146088,1971.976320,3316.113281,-0.039186,0.078125,...,-5.223142,-0.076858,-6.150883,-3.480076,-5.094445,0.202235,-4.552980,0.501928,4,rock
